In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple, List
from statistics import median, mean
import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_rows", 500)

In [4]:
df_month_windows_1 = pd.read_csv('data/df_month_windows_1.csv', parse_dates=["date"])
df_month_windows_3 = pd.read_csv('data/df_month_windows_3.csv', parse_dates=["date"])
df_week_windows_1 = pd.read_csv('data/df_week_windows_1.csv', parse_dates=["date"])
df_week_windows_6 = pd.read_csv('data/df_week_windows_6.csv', parse_dates=["date"])
# transactions['date'] = pd.to_datetime(transactions['date'])
# users['update_date'] = pd.to_datetime(users['update_date'])

In [24]:
print(df_month_windows_1.shape)
print(df_month_windows_3.shape)
print(df_week_windows_1.shape)
print(df_week_windows_6.shape)
print(df_month_windows_1.columns)
final_dataframes = {"month1": df_month_windows_1, "month3": df_month_windows_3, "week1": df_week_windows_1, "week6": df_week_windows_6}
# print(final_dataframes)

(2555, 27)
(3441, 27)
(8711, 47)
(14757, 47)
Index(['date', 'amount', 'amount_other', 'balance', 'account_id',
       'business_NAF_code', 'user_id', 'month_sin', 'month_cos',
       'previous_balance_1', 'previous_amount_1', 'previous_other_1',
       'previous_balance_2', 'previous_amount_2', 'previous_other_2',
       'previous_balance_3', 'previous_amount_3', 'previous_other_3',
       'previous_balance_4', 'previous_amount_4', 'previous_other_4',
       'previous_balance_5', 'previous_amount_5', 'previous_other_5',
       'previous_balance_6', 'previous_amount_6', 'previous_other_6'],
      dtype='object')


In [27]:
def _error(actual: np.ndarray, predicted: np.ndarray):
    """ Simple error """
    return actual - predicted

def mae(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Absolute Error """
    return np.mean(np.abs(_error(actual, predicted)))

def mse(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Squared Error """
    return np.mean(np.square(_error(actual, predicted)))

def rmse(actual: np.ndarray, predicted: np.ndarray):
    """ Root Mean Squared Error """
    return np.sqrt(mse(actual, predicted))


In [28]:
def remove_features(df, keywords):
    to_remove = []
    for col in df.columns:
        for keyword in keywords:
            if keyword in col:
                to_remove.append(col)
            
    df = df.drop(to_remove, axis=1)
    return df

Looking at the problem, we don't have much previous data to infer from, either 6 or 12 data points. It means that, if possible, we need to make use of all the information we have, that means

Since I'm not proficient on this type of problem and data, and I want to learn more about it, I'm going to try out a lot of different things.

Here I built some very basic models to compare our more advanced models to. I experimented with multiple combinations of features like adding the other_amount, the balance, one-hot-encoding the NAF code etc. The best results were with just previous amounts and balances.

I chose Mean Absolute Error as a metric for its simplicity and interpretability.

In [58]:
import sklearn.metrics as mtr
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from statsmodels.api import OLS
from statsmodels.api import add_constant

def run_basic_models(dataframes, features_to_remove):
    results_dataframe = pd.DataFrame(columns=
        ["Naive", "Seasonal Naive", "Average", "Median", "Drift", "OLS_Regression"],
        index=dataframes.keys())
    i=0
    for name, df in dataframes.items():

#         df = pd.get_dummies(df, drop_first=True)

        total_drifts_mae = []
        ols_mae = []
        naive_mae = []
        average_mae = []
        median_mae = []
        naive_seasonal_mae = []
        X = df.loc[:, df.columns != 'amount']
        y = df["amount"]
        
        X = remove_features(X, ["other"])
        
        kf = KFold(n_splits=5)

        for train_index, test_index in kf.split(X,y):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            X_train = X_train.drop([column for column in X_train.columns if column in features_to_remove], axis=1) # drop columns not in features
            X_test = X_test.drop([column for column in X_test.columns if column in features_to_remove], axis=1) 

            drifts_mae = []
            previous_amount_columns = [column for column in df.columns if "previous_amount" in column]
            previous_amount_balance_columns = [column for column in df.columns if "previous_amount" in column or "previous_balance" in column]

            
            
            # Naive Forecast (last value)
            NaiveForecast = X_train['previous_amount_1']
            naive_mae.append(mae(y_train, NaiveForecast))


            # Average Forecast (average of all previous values)
            average_forecast = X_train[previous_amount_columns].mean(axis=1)
            average_mae.append(mae(y_train, average_forecast))

            # Median Forecast (median of all previous values)
            median_forecast = X_train[previous_amount_columns].median(axis=1)
            median_mae.append(mae(y_train, median_forecast))

            # Seasonal Naive Forecast (predict using value from 4 weeks ago for weeks, and just normal naive for month)
            if "week" in name:
                NaiveSeasonalForecast = X_train['previous_amount_4']
                naive_seasonal_mae.append(mae(y_train, NaiveSeasonalForecast))
            else:
                NaiveSeasonalForecast = X_train['previous_amount_1']
                naive_seasonal_mae.append(mae(y_train, NaiveSeasonalForecast))


            # Drift Forecast (drawing a line from the first to the last observations and extrapolating from it)
            row_count = 0
            for idx, row in X_train.iterrows():
                true_value = y_train.iloc[row_count]
                y_t = row['previous_amount_1'] # last amount
                m = (y_t - row[previous_amount_columns[-1]]) / len(previous_amount_columns)
                h = np.linspace(0,len(previous_amount_columns)-1, len(previous_amount_columns))
                drift = y_t + m * h
                drifts_mae.append(mae(true_value, drift))
                row_count+=1

            total_drifts_mae.append(mean(drifts_mae))


            # OLS Linear Regression (with previous values as regressors, so like an autoregression)
#             X_train = add_constant(X_train)

            model = OLS(y_train, X_train)
            
            model_fit = model.fit()
            model_fit.save(name+".pickle")
            X_test_ols =  X_test
            ols_pred = model_fit.predict(X_test_ols)
            median_X_test = X_test[previous_amount_columns].median(axis=1)
            if "month" in name:
                ols_pred = np.array([x if x > 0 else median_X_test.iloc[i] for i, x in enumerate(ols_pred)])
            else:
                ols_pred = np.array([x if x < 0 else median_X_test.iloc[i] for i, x in enumerate(ols_pred)])
            ols_mae.append(mae(ols_pred, y_test))




        results_dataframe.iloc[i] = [mean(naive_mae), mean(naive_seasonal_mae), mean(average_mae), mean(median_mae), mean(total_drifts_mae), mean(ols_mae)]
        i+=1


    print(X_train.columns) # to see which features were used
    print(results_dataframe)
    return results_dataframe

In [59]:
only_numerical_features = ["id", "date", "user_id", "account_id", "business_NAF_code", "amount_other", "balance"]
only_prices = ["id", "date", "user_id", "account_id", "business_NAF_code", 'month_sin', 'month_cos', 'month_week_sin', 'month_week_cos', "amount_other", "balance"]

r = run_basic_models(final_dataframes, only_prices)

Index(['previous_balance_1', 'previous_amount_1', 'previous_balance_2',
       'previous_amount_2', 'previous_balance_3', 'previous_amount_3',
       'previous_balance_4', 'previous_amount_4', 'previous_balance_5',
       'previous_amount_5', 'previous_balance_6', 'previous_amount_6',
       'previous_balance_7', 'previous_amount_7', 'previous_balance_8',
       'previous_amount_8', 'previous_balance_9', 'previous_amount_9',
       'previous_balance_10', 'previous_amount_10', 'previous_balance_11',
       'previous_amount_11', 'previous_balance_12', 'previous_amount_12'],
      dtype='object')
              Naive Seasonal Naive      Average       Median        Drift  \
month1  3374.712689    3374.712689  3039.817520  2982.363331  4510.578097   
month3  3326.056145    3326.056145  3074.164159  3080.359561  4509.242378   
week1   1181.301255    1075.108392   931.651462   881.639308  1591.123262   
week6   1079.848293     983.399735   863.732649   811.757910  1468.490042   

       OLS_Re

Results look better when using less filtered datasets, but that's maybe just cause it's easier to be closer to the truth when the truth is often 0. The more filtered dataset contains less repetitive data, so maybe it could make sense to use it, especially because new data coming in will be last 12 weeks of a least 6 months history, so we're more less likely to have very few transactions.
                
Regression has the best results, let's try to beat that with more complex models.

I decided to try a Random Forest Regressor, for its robustness to overfitting and outliers.

In [10]:
# df_month_windows_1=df_month_windows_1.iloc[:100]
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
def gridsearchRFR(df, param_grid):
    df = pd.get_dummies(df, drop_first=True) # for categorical data like naf_code
    account_ids = pd.Series(df["account_id"].unique())
    test_account_ids = account_ids.sample(frac=0.2, random_state=22)

    df_test = df[df["account_id"].isin(test_account_ids)]
    df_train = df[~df["account_id"].isin(test_account_ids)]


    X_train = df_train.drop(["date", "user_id", "account_id", "amount", "balance", "amount_other"], axis=1)
    y_train = df_train["amount"]
    X_test = df_test.drop(["date", "user_id", "account_id", "amount", "balance", "amount_other"], axis=1)
    y_test = df_test["amount"]


    gsc = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid=param_grid,
        cv=5, scoring='neg_mean_absolute_error', verbose=2, n_jobs=-1)

    grid_result = gsc.fit(X_train, y_train)
    best_params = grid_result.best_params_
    print(best_params)

    min_samples_split = best_params.get("min_samples_split", 2)
    min_samples_leaf = best_params.get("min_samples_leaf", 1)
    max_features = best_params.get("max_features", "auto")
    rfr = RandomForestRegressor(
        max_depth=best_params["max_depth"], 
        n_estimators=best_params["n_estimators"],
        min_samples_split = min_samples_split,
        min_samples_leaf = min_samples_leaf,
        max_features = max_features,
        random_state=False, verbose=1)

    rfr.fit(X_train, y_train)
    predictions = rfr.predict(X_test)

    mae_score = mae(predictions, y_test)
    return mae_score

param_grid = {
            'max_depth': range(4,8),
            'n_estimators': (100, 200, 500, 1000),
        }
print("month results")
mae_score_m3 = gridsearchRFR(df_month_windows_3, param_grid)
print(mae_score_m3)

print("week results")
mae_score_w6 = gridsearchRFR(df_week_windows_6, param_grid)
print(mae_score_w6)


month results
Fitting 5 folds for each of 16 candidates, totalling 80 fits
{'max_depth': 7, 'n_estimators': 500}


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   36.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.1s finished


2571.100233146333
week results
Fitting 5 folds for each of 16 candidates, totalling 80 fits
{'max_depth': 7, 'n_estimators': 1000}


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 1000


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  2.9min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


816.0879575950358


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished


Result is 2571 Mean Absolute Error. Better than the simple models above. The one-hot encoded naf_code adds around 80 features. One-hot encoding categorical variables with high cardinality can cause inefficiency in tree-based ensembles. Continuous variables will be given more importance than the dummy variables by the algorithm which will obscure the order of feature importance resulting in poorer performance.

I chose to remove them, and to tune the model's hyperparameters with a cross-validated random search.

In [81]:
from sklearn.model_selection import RandomizedSearchCV

def randomsearchRFR(df):
    account_ids = pd.Series(df_month_windows_3["account_id"].unique())
    test_account_ids = account_ids.sample(frac=0.2, random_state=22)

    df_test = df_month_windows_3[df_month_windows_3["account_id"].isin(test_account_ids)]
    df_train = df_month_windows_3[~df_month_windows_3["account_id"].isin(test_account_ids)]
    
    

    X_train = df_train.drop(["date", "user_id", "account_id", "amount", "balance", "amount_other", "business_NAF_code"], axis=1)
    y_train = df_train["amount"]
    X_test = df_test.drop(["date", "user_id", "account_id", "amount", "balance", "amount_other", "business_NAF_code"], axis=1)
    y_test = df_test["amount"]


    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 40, stop = 300, num = 10)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(50, 90, num = 10)]
#     max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 4, 6, 8, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4, 6, 8]

    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf}


    rsc = RandomizedSearchCV(
        estimator=RandomForestRegressor(),
        param_distributions =random_grid,
        n_iter = 100,
        random_state=22,
        cv=5, scoring='neg_mean_absolute_error', verbose=2, n_jobs=-1)

    rsc_result = rsc.fit(X_train, y_train)
    best_params = rsc_result.best_params_

    print(best_params)

    rfr = RandomForestRegressor(
        **best_params,
        random_state=False, verbose=1)

    rfr.fit(X_train, y_train)
    predictions = rfr.predict(X_test)
    mae_score = mae(predictions, y_test)
    
    return rsc_result, rfr, mae_score


In [82]:
rsc_result, rs_rfr, mae_score_m3 = randomsearchRFR(df_month_windows_3)
print(mae_score_m3)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'n_estimators': 271, 'min_samples_split': 2, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'max_depth': 76}


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


2164.8617681465385


[Parallel(n_jobs=1)]: Done 271 out of 271 | elapsed:    2.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 271 out of 271 | elapsed:    0.0s finished


In [88]:
print(rsc_result.cv_results_['mean_test_score'])

[-2979.24635306 -2974.57155658 -2962.22097241 -2953.51445007
 -2965.19141097 -2993.70393031 -2964.93680678 -2969.43343227
 -2955.1672958  -2985.87159894 -2996.97296996 -3003.94295584
 -2975.10163439 -3003.50320342 -2967.92920718 -3000.74352308
 -2960.72206283 -2999.29870994 -2969.86426744 -2996.00650246
 -2959.4072583  -2958.62599803 -3036.70453474 -2971.3043492
 -2968.71750405 -2970.89064315 -2984.5195747  -2971.8444927
 -2959.34429866 -2961.0661957  -2966.13759264 -2964.4120373
 -2967.31182314 -2961.59342761 -2989.02628828 -3028.72547064
 -2967.81047544 -3030.09952766 -2961.45998069 -2979.14243541
 -2964.93299614 -2969.61626225 -2969.9951     -2954.0606751
 -2955.94745643 -2967.68609487 -3002.00203447 -2985.64331292
 -2968.33444856 -2968.57286291 -2967.22102821 -2960.54014162
 -2982.35840864 -2987.47999571 -3035.32095169 -2977.42693988
 -3046.5455253  -2956.61179408 -2960.44096506 -2967.22959265
 -2969.24238121 -3004.77507383 -3013.32694135 -3076.10092853
 -3036.97872809 -2956.397209

This time the result is 2164, much better. But we can clearly see that changing the parameters resulted in virtually no improvement on the mean test score of all 5 cross validation. I was planning on doing grid search around the best random search parameters, but it seems irrelevant now. The Random Forest Regression is clearly underfitting and ineffective.

Let's just try training a new model with those parameters but with a different sampling random state so we don't train and evaluate on the same accounts.

In [89]:
# params = {'max_depth': 70, 'min_samples_leaf': 6, 'min_samples_split': 7, 'n_estimators': 300}
params = {'n_estimators': 271, 'min_samples_split': 2, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'max_depth': 76}

account_ids = pd.Series(df_month_windows_3["account_id"].unique())
test_account_ids = account_ids.sample(frac=0.2, random_state=21)

df_test = df_month_windows_3[df_month_windows_3["account_id"].isin(test_account_ids)]
df_train = df_month_windows_3[~df_month_windows_3["account_id"].isin(test_account_ids)]

X_train = df_train.drop(["date", "user_id", "account_id", "amount", "balance", "amount_other", "business_NAF_code"], axis=1)
y_train = df_train["amount"]
X_test = df_test.drop(["date", "user_id", "account_id", "amount", "balance", "amount_other", "business_NAF_code"], axis=1)
y_test = df_test["amount"]



rfr = RandomForestRegressor(
        **params,
        random_state=False, verbose=1, n_jobs=-1)

rfr.fit(X_train, y_train)
predictions = rfr.predict(X_test)

mae_score = mae(predictions, y_test)
print(mae_score)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 271 out of 271 | elapsed:    0.6s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s


2694.9355614583524


[Parallel(n_jobs=12)]: Done 271 out of 271 | elapsed:    0.0s finished


The result is much worse. Let's try with random sampling over the index and not the accounts.

In [91]:
params = {'n_estimators': 271, 'min_samples_split': 2, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'max_depth': 76}

test_ids = df_month_windows_3.sample(frac=0.2, random_state=22).index

df_test = df_month_windows_3.iloc[test_ids].sort_index()
df_train = df_month_windows_3.loc[~df_month_windows_3.index.isin(test_ids)]


X_train = df_train.drop(["date", "user_id", "account_id", "amount", "balance", "amount_other", "business_NAF_code"], axis=1)
y_train = df_train["amount"]
X_test = df_test.drop(["date", "user_id", "account_id", "amount", "balance", "amount_other", "business_NAF_code"], axis=1)
y_test = df_test["amount"]



rfr = RandomForestRegressor(
        **params,
        random_state=False, verbose=1, n_jobs=-1)

rfr.fit(X_train, y_train)
predictions = rfr.predict(X_test)

mae_score = mae(predictions, y_test)
print(mae_score)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 271 out of 271 | elapsed:    0.7s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 271 out of 271 | elapsed:    0.0s finished


2716.92814881773


Again, mediocre results. I could try to refine the hyperparameters even more by plotting each parameter's effect on the score but I want to try another approach that I think could be more effective.

After doing more research on panel data and how to model it and forecast with it, I have learned about fixed effects, random effects, and mixed models.

After reading everything, it seems like the most promising way to have a suitable model to forecast monthly incomes across multiple accounts. It models the relationships over time but also across groups.

In [51]:
df_month = pd.read_csv('data/df_monthly_data.csv', parse_dates=["date"])
df_week = pd.read_csv('data/df_weekly_data.csv', parse_dates=["date"])

I tried modeling monthly income according to balance and monthly expense, grouped by account_id, then grouped by NAF_code, which showed better results. It doesn't allow to forecast but it showed good relationship between those variables grouped by naf code.

In [101]:
# min and max number of month data for an account
print(max(df_month.groupby("account_id")['date'].count().values))
print(min(df_month.groupby("account_id")['date'].count().values))

# let's delete accounts which have less than 7 months of data available
idx_to_delete = []
for idx, count in enumerate(df_month.groupby("account_id")['date'].count()):
    if count < 7:
        idx_to_delete.append(idx)

account_ids = df_month.groupby("account_id")['date'].count().index

account_ids_to_delete = [account_ids[id_to_delete] for id_to_delete in idx_to_delete]
print(account_ids_to_delete)
df_month = df_month.loc[~df_month["account_id"].isin(account_ids_to_delete)]

33
1
[126, 201, 333]


In [113]:
# make dates universal accross accounts (0 for first month of data and increment by 1 after)
account_dfs=[]
for _, df_account in df_month.groupby("account_id"):
    df_account = df_account.sort_values(by="date", ascending=False)
    df_account["date"] = range(0, df_account.shape[0])
    
    account_dfs.append(df_account.sort_values(by="date"))

df_month = pd.concat(account_df for account_df in account_dfs)
print(df_month.loc[df_month["amount"]==0].shape)

(204, 9)


In [132]:
from linearmodels.panel import PooledOLS
import statsmodels.api as sm
from linearmodels.panel import RandomEffects
from linearmodels.panel import PanelOLS
from statsmodels.regression.mixed_linear_model import MixedLM

pred_results = []
models=[]

for i in range(10, 15):
    account_ids = pd.Series(df_month["account_id"].unique())
    test_account_ids = account_ids.sample(frac=0.2, random_state=i)

    df_test = df_month[df_month["account_id"].isin(test_account_ids)]
    df_train = df_month[~df_month["account_id"].isin(test_account_ids)]

    
    y_test = df_test["amount"]
    
    model = sm.MixedLM.from_formula("amount ~ balance + amount_other", df_train, groups=df_train["business_NAF_code"])
    
    result = model.fit()
    models.append(result)
    predictions = result.predict(df_test)
    pred_result = mae(predictions, y_test)
    
    print(pred_result)
    pred_results.append(pred_result)

2539.7481653927584
2037.5380132287037
1757.1153673261424
1591.704749543777
2133.8986144334885


In [120]:
df_month["mean_balance"] = df_month.groupby('account_id')['balance'].transform('mean')
df_month["previous_amount"] = df_month["amount"].shift(1).fillna(method='bfill')
df_month["previous_balance"] = df_month["balance"].shift(1).fillna(method='bfill')
df_month["previous_amount_other"] = df_month["amount_other"].shift(1).fillna(method='bfill')

In [146]:
pred_results = []
models=[]

for i in range(23, 28):
    account_ids = pd.Series(df_month["account_id"].unique())
    test_account_ids = account_ids.sample(frac=0.2, random_state=i)

    df_test = df_month[df_month["account_id"].isin(test_account_ids)]
    df_train = df_month[~df_month["account_id"].isin(test_account_ids)]
    
    
    y_test = df_test["amount"]
    
    model = sm.MixedLM.from_formula("amount ~ previous_amount + previous_balance + previous_amount_other", df_train, groups=df_train["business_NAF_code"])
    
    result = model.fit()
    models.append(result)
    predictions = result.predict(df_test)
    pred_result = mae(predictions, y_test)
    
    print(pred_result)
    pred_results.append(pred_result)

3729.558906025532
3638.284668221423
3306.530458812592
3225.356325702513
3502.013671698812


The model, as could be expected, performs poorly when only using previous month data. Let's use the 6 previous months data with our windows dataset, to see if a Mixed Linear Model could perform well and make forecasts.

In [201]:
pred_results = []
models=[]

for i in range(1, 6):
    account_ids = pd.Series(df_month_windows_3["account_id"].unique())
    test_account_ids = account_ids.sample(frac=0.2, random_state=i)

    df_test = df_month_windows_3[df_month_windows_3["account_id"].isin(test_account_ids)]
    df_train = df_month_windows_3[~df_month_windows_3["account_id"].isin(test_account_ids)]
    
    previous_amount_balance_columns = [column for column in df_train.columns if "previous" in column]

    
    X_train = sm.add_constant(df_train[previous_amount_balance_columns])
    y_train = np.asarray(df_train["amount"])
    X_test = sm.add_constant(df_test[previous_amount_balance_columns])
    y_test = df_test["amount"]

    model = sm.MixedLM(y_train, X_train, groups=df_train["account_id"])
    result = model.fit()
    models.append(result)
    predictions = result.predict(X_test)
    pred_result = mae(predictions, y_test)
    print(pred_result)
    pred_results.append(pred_result)

2785.4572625055544
3686.4419098096387
3174.464343712196
2978.794126172
2982.023450924204


In [136]:
for model in models:
    print(model.summary())

print(pred_results)

                   Mixed Linear Model Regression Results
Model:                  MixedLM      Dependent Variable:      y            
No. Observations:       2664         Method:                  REML         
No. Groups:             75           Scale:                   27863210.6569
Min. group size:        2            Log-Likelihood:          -26621.4523  
Max. group size:        416          Converged:               No           
Mean group size:        35.5                                               
---------------------------------------------------------------------------
                     Coef.    Std.Err.    z    P>|z|    [0.025     0.975]  
---------------------------------------------------------------------------
const                525.658    140.645  3.737 0.000     249.998    801.317
previous_balance_1    -1.782 275415.138 -0.000 1.000 -539805.533 539801.970
previous_amount_1      1.796 275415.138  0.000 1.000 -539801.955 539805.547
previous_other_1       1.231 27

              Mixed Linear Model Regression Results
Model:               MixedLM   Dependent Variable:   y            
No. Observations:    2791      Method:               REML         
No. Groups:          77        Scale:                27509552.7194
Min. group size:     2         Log-Likelihood:       -27871.7716  
Max. group size:     368       Converged:            No           
Mean group size:     36.2                                         
------------------------------------------------------------------
                     Coef.   Std.Err.   z    P>|z|  [0.025  0.975]
------------------------------------------------------------------
const                547.471  131.421  4.166 0.000 289.890 805.051
previous_balance_1     2.170                                      
previous_amount_1     -2.133                                      
previous_other_1      -2.655                                      
previous_balance_2    -6.729                                      
previous_a

In [202]:
pred_results = []
models=[]

for i in range(1, 6):
    account_ids = pd.Series(df_week_windows_6["account_id"].unique())
    test_account_ids = account_ids.sample(frac=0.2, random_state=i)

    df_test = df_week_windows_6[df_week_windows_6["account_id"].isin(test_account_ids)]
    df_train = df_week_windows_6[~df_week_windows_6["account_id"].isin(test_account_ids)]
    
    previous_amount_balance_columns = [column for column in df_train.columns if "previous" in column]

    
    X_train = sm.add_constant(df_train[previous_amount_balance_columns])
    y_train = np.asarray(df_train["amount"])
    X_test = sm.add_constant(df_test[previous_amount_balance_columns])
    y_test = df_test["amount"]

    model = sm.MixedLM(y_train, X_train, groups=df_train["account_id"])
    result = model.fit()
    models.append(result)
    predictions = result.predict(X_test)
    pred_result = mae(predictions, y_test)
    print(pred_result)
    pred_results.append(pred_result)

845.2194032526189
1258.0980514523353
986.5145460552178
704.1772454954811
779.2934281032846


Results are not bad but on average worse than OLS regression. This type of model is made to model the variance between groups and among groups, using the variable that change within groups (amount, balance) and the variables that change between groups (naf_code). But unlike the references I read online, our forecast variable and our predictor variables are correlated, and we have very little data (6 or 12 points). I don't know enough about this field to create something more advanced with fixed/random/mixed effects.

But in the meantime I have read about Vector Auto-Regression (VAR). In the VAR framework, all variables are treated symmetrically. They are all modelled as if they all influence each other equally.

To use VAR, we need our time series to be stationary, after some tries, 33% of data couldn't reach stationarity (when using only amount and other amount, 40% when using balance), even when trying many differencing (1 to 5). And even then, the lack of data forced me to have a maximum of 3 lags for VAR, meaning it could only forecast using data from 1-3 weeks ago, and sometimes 0. So obviously it was not usable. I think this would have been a really good option if we could use more data.

In [ ]:
from statsmodels.tsa.api import VAR
from statsmodels.stats.stattools import durbin_watson

non_stationary_ids = []
all_used_lags = {}
amount_columns = [column for column in df_week_windows_6.columns if "previous_amount" in column]
other_columns = [column for column in df_week_windows_6.columns if "previous_other" in column]
results = []
# for variable in ["amount", "other"]:
used_lags = {}
variable_columns = [column for column in df_week_windows_6.columns if "previous_"+variable in column]
for idx, data in df_week_windows_1.iterrows():
    amount_data = data[amount_columns].iloc[::-1]
    other_data = data[other_columns].iloc[::-1]
    
    for i in range(1, 5):
        diff_amount_data = amount_data.diff(i).fillna(0)
        diff_other_data = other_data.diff(i).fillna(0)

        if diff_amount_data.shape[0]>0 and diff_other_data.shape[0]>0:
            amount_p_value = sm.tsa.stattools.adfuller(diff_amount_data)[1]
            other_p_value = sm.tsa.stattools.adfuller(diff_other_data)[1]

            if amount_p_value < 0.05 and other_p_value < 0.05:
                used_lag = i
                used_lags[idx] = used_lag
                
                df_data = pd.DataFrame(list(zip(diff_amount_data, diff_other_data)), columns=["amount", "other_amount"])

                var = VAR(df_data)
                lag_results = var.fit(maxlags=3, ic='aic')
                lag_order = lag_results.k_ar
    
                if lag_order > 0:
                    predictions = lag_results.forecast(df_data.values[-lag_order:], 1)
                    result = mae(predictions[0][0], df_week_windows_6.iloc[idx]["amount"])
                    results.append(result)
                break
                
            else:
                non_stationary_ids.append(idx)

            

all_used_lags[variable] = used_lags
print(np.mean(results))
print((len(set(non_stationary_ids))/df_week_windows_6.shape[0])*100, "% of the data is non stationary")

Purely for the sake of experimenting, I tried some classic econometrics models that work with seasonality, on the weekly expenses data.

In [57]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
amount_columns = [column for column in df_week_windows_1.columns if "previous_amount" in column]
y_test = []
predictions = []
for idx, row in df_week_windows_1.iterrows():
    data = row[amount_columns].iloc[::-1].values
    amount = row["amount"]
    # create class
    model = ExponentialSmoothing(data, seasonal_periods = 4)
    # fit model
    model_fit = model.fit()
    # make prediction
    y_hat = model.predict(model_fit.params)
    
    y_test.append(amount)
    predictions.append(y_hat)
    
print(mae(np.array(y_test), np.array(predictions)))

1554.2764758538742


In [8]:
import itertools
import statsmodels.api as sm


### Run Grid Search for SARIMAX parameters (this code will take a while to run)


def sarimax_gridsearch(ts, pdq, pdqs, dates, maxiter=50, metric="aic"):

    # Run a grid search with pdq and seasonal pdq parameters and get the best AIC value
    ans = []
    for comb in pdq:
        for combs in pdqs:
            try:
                mod = sm.tsa.statespace.SARIMAX(ts,
                                                order=comb,
                                                seasonal_order=combs,
                                                enforce_stationarity=False,
                                                enforce_invertibility=False,
                                                dates=dates)

                output = mod.fit(maxiter=maxiter)
                
                if metric == "bic":
                    ans.append([comb, combs, output.bic])
                elif metric == "mae":
                    ans.append([comb, combs, output.mae])
                else:
                    ans.append([comb, combs, output.aic])

            except Exception as e:
                raise e
                continue
            
    # Find the parameters with minimal AIC value

    ans_df = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'mae'])

    # Sort and return top 5 combinations
    ans_df = ans_df.sort_values(by=['mae'],ascending=True)[0:5]
    
    return ans_df



In [9]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from random import random
import itertools
import statsmodels.api as sm

amount_columns = [column for column in df_week_windows_6.columns if "previous_amount" in column]
y_test = []
predictions_mae = []
predictions_aic = []
best_results_mae = []
best_results_aic = []
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))
pdqs = [(x[0], x[1], x[2], 4) for x in list(itertools.product(p, d, q))]


df_sample = df_week_windows_1.sample(50, random_state = 42)

for idx, row in df_sample.iterrows():
    data = np.array(row[amount_columns].iloc[::-1].values).astype('float')
    amount = row["amount"]
    y_test.append(amount)
    dates = []
    for i in range(1, 13):
        i_date = row['date'] - pd.DateOffset(weeks=i)
        dates.append(i_date)
    
    dates = list(reversed(dates))
    results_mae = sarimax_gridsearch(data, pdq, pdqs, dates=dates, metric="bic")

    fit_pdq_mae = results_mae.iloc[0]["pdq"]
    fit_pdqs_mae = results_mae.iloc[0]["pdqs"]
    
    best_results_mae.append(results_mae)
#     print(idx)
#     print(data)
#     print(results_mae)
    model_mae = SARIMAX(data, order=fit_pdq_mae, seasonal_order=fit_pdqs_mae, initialization='approximate_diffuse')
    model_fit_mae = model_mae.fit(disp=False)

    y_hat_mae = model_fit_mae.forecast(1)[0]
#     print(y_hat)
#     print("\n\n")
    
    
    predictions_mae.append(y_hat_mae)
    
    results_aic = sarimax_gridsearch(data, pdq, pdqs, dates=dates, metric="mae")

    fit_pdq_aic = results_aic.iloc[0]["pdq"]
    fit_pdqs_aic = results_aic.iloc[0]["pdqs"]
    
    best_results_aic.append(results_aic)

    model_aic = SARIMAX(data, order=fit_pdq_aic, seasonal_order=fit_pdqs_aic, initialization='approximate_diffuse')
    model_fit_aic = model_aic.fit(disp=False)

    y_hat_aic = model_fit_aic.forecast(1)[0]

    
    predictions_aic.append(y_hat_aic)
    
    
print(mae(np.array(y_test), np.array(predictions_aic)))
print(mae(np.array(y_test), np.array(predictions_mae)))

# print(best_results_mae)
# print(best_results_aic)

1847.9700426609515
2930.2787383240975


Last, I tried using Facebook's Prophet tool for forecasting. It is supposed to work well even with few data. It has support for specifying seasonality, but after trial and error I found that it performs better with nothing. (I also tried on the monthly data but it had poor results)

In [52]:
# delete accounts who have less than 13 weeks of data (12 for training and 1 to test)
idx_to_delete = []
for idx, count in enumerate(df_week.groupby("account_id")['date'].count()):
    if count < 13:
        idx_to_delete.append(idx)


account_ids = df_week.groupby("account_id")['date'].count().index

account_ids_to_delete = [account_ids[id_to_delete] for id_to_delete in idx_to_delete]
print(account_ids_to_delete)
df_week = df_week.loc[~df_week["account_id"].isin(account_ids_to_delete)]

[126, 333]


In [53]:
# get 13 weeks of data per account
account_dfs=[]
for _, df_account in df_week.groupby("account_id"):
    df_account = df_account.sort_values(by="date", ascending=True)
#     df_account["date"] = range(0, df_account.shape[0])
    
    account_dfs.append(df_account.sort_values(by="date", ascending=False).iloc[:13])

df_week = pd.concat(account_df for account_df in account_dfs)

In [54]:
from fbprophet import Prophet
import logging, sys

logging.disable(sys.maxsize)

predictions = []
y_true = []
for account_df in account_dfs:
    account_df = account_df[["date", "amount"]]
    account_df.columns = ["ds", "y"]
    
#     print("ree\n")
    x = account_df.iloc[1:].sort_values(by="ds", ascending=True)
    y = account_df.iloc[0][1]
#     print(x)
    m = Prophet()

    m.fit(x)
    future = m.make_future_dataframe(periods=1, freq='W')
    forecast = m.predict(future)
    prediction = forecast[["yhat"]].iloc[-1][0]
    
    if prediction > 0:
        prediction = np.median(x["y"])
        
    predictions.append(prediction)
    y_true.append(y)

score = mae(np.array(predictions), np.array(y_true))

In [55]:
print(score)

961.2123421299352


Not bad results, but still worse than our linear regression. I am sure that with well thought out feature engineering and a simple regression we could have had better results. I have learned a TON of things about time series modeling and forecasting. I am disappointed I could not get better results with them, but it seems that the goal of this exercise was to see the thought process, rather than the actual result.

My mistake was that at first, I thought more complex techniques, that are able to model more exogenous variables and/or seasonality, would perform better. I probably should have realised that our low number of observations would make complex models too inefficient.